<a href="https://colab.research.google.com/github/namanh218/vietnamese_rec/blob/master/CTC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install mahotas

In [0]:
!pip install tensorflow==2.1.0rc1

In [0]:
#Import library
import shutil
import pathlib
import cv2
import numpy as np
from google.colab.patches import cv2_imshow #only in Colab because using cv2.imshow is not allowed in Colab
import os
import mahotas
import imutils
import tensorflow as tf
import json
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [0]:
 #Check version Tensorflow
 print(tf.__version__)

In [0]:
#Access to drive
from google.colab import drive
drive.mount('/content/drive')

In [0]:
#All path to access
img_train_path = './drive/My Drive/0916_Data Samples 2'
img_test_path = './drive/My Drive/1015_Private Test'
img_test_folder = pathlib.Path(img_test_path)
img_folder = pathlib.Path(img_train_path)

In [0]:
#Path to json file
json_name = 'labels.json'
path = os.path.join(img_train_path, json_name)
labels = json.loads(open(path).read())
test_path_2 = os.path.join(img_test_path, json_name)
test_labels = json.loads(open(test_path_2).read())

In [0]:
#Func to get path
def sep_json(list_, dict_, path_):
  for key, value in dict_.items():
    path = os.path.join(path_, key)
    list_.append(path)
  return list_

In [0]:
#Get all path from train images
all_train_path = []
sep_json(all_train_path, labels, img_train_path)

In [0]:
#Get all path from test images
all_test_path = []
sep_json(all_test_path, test_labels, img_test_path)

In [0]:
#Delete json link
def delete_json(list_, path_):
  for i in range(len(list_)):
    if path_.strip('.').lstrip('/') + '/labels.json' == list_[i]:
      print(i)
      list_.pop(i)

delete_json(all_train_path, img_train_path)
delete_json(all_test_path, img_test_path)      

In [0]:
#All the letter and function to pad sequences letters
letters = " #'()+,-./:0123456789ABCDEFGHIJKLMNOPQRSTUVWXYabcdeghiklmnopqrstuwvxyzÂÊÔàáâãèẹéêìíòóôõùúýăĐđĩũƠơưạảấầẫẩậắằẵặẻẽếềểễệỉịọỏốồổỗộớờởỡợụủỨứừửữựỳỵỷỹ"
number_letters = len(letters)
print(number_letters)
def text_to_labels(text):
  for c in text:
    if letters.find(c) == -1:
      print(c)
  return list(map(lambda x: letters.index(x), text))

def label_to_text(labels):
    return ''.join(list(map(lambda x: letters[x] if x < len(letters) else "", labels)))

def map_letters(list_key, list_value, map_, dict_):
  for key, value in dict_.items():
    list_key.append(key)
    list_value.append(value)
    label = text_to_labels(value)
    label = pad_sequences([label], maxlen = 70, padding ='post')
    map_.append(label)    

In [0]:
#Map for the train
train_key = []
train_values = []
train_labels_map = []
map_letters(train_key, train_values, train_labels_map, labels)

In [0]:
#Map for the test
test_key = []
test_values = []
test_labels_map = []
map_letters(test_key, test_values, test_labels_map, test_labels)

In [0]:
#Preprocess data
def load_and_preprocess_from_path_label(path, label):
  return preprocess_path(path), label
def preprocess_path(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=1)
    # Use `convert_image_dtype` to convert to floats in the [0,1] range.
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, [2048, 128])
    image /= 255.
    # image = (image*2) - 1  # normalize to [-1,1] range
    # image = tf.image.per_image_standardization(image)
    return image

In [0]:
#Prepare for dataset
BATCH_SIZE = 32
TRAIN_SAMPLES = 1823
TEST_SAMPLES = 549
AUTOTUNE = tf.data.experimental.AUTOTUNE

def prepare_for_training(ds, cache=True, shuffle_buffer_size=100, augment=False):
    # This is a small dataset, only load it once, and keep it in memory.
    # use `.cache(filename)` to cache preprocessing work for datasets that don't
    # fit in memory.
    if cache:
        if isinstance(cache, str):
            ds = ds.cache(cache)
        else:
            ds = ds.cache()

    ds = ds.shuffle(buffer_size=shuffle_buffer_size)

    # Repeat forever
    ds = ds.repeat()

    ds = ds.batch(BATCH_SIZE)
    
    # if augment:
    #     ds.map(augmentation, num_parallel_calls=AUTOTUNE)

    # `prefetch` lets the dataset fetch batches in the background while the model
    # is training.
    ds = ds.prefetch(buffer_size=AUTOTUNE)

    return ds

In [0]:
#Create new dataset
train_ds = tf.data.Dataset.from_tensor_slices((all_train_path, train_labels_map))
test_ds = tf.data.Dataset.from_tensor_slices((all_test_path, test_labels_map))

In [0]:
#Prepare for train set
train_ds = train_ds.map(load_and_preprocess_from_path_label, num_parallel_calls=AUTOTUNE)
train_ds = prepare_for_training(train_ds, shuffle_buffer_size=TRAIN_SAMPLES)

In [0]:
#Prepare for test set
test_ds = test_ds.map(load_and_preprocess_from_path_label, num_parallel_calls=AUTOTUNE)
test_ds = prepare_for_training(test_ds, cache=False, shuffle_buffer_size=TEST_SAMPLES)

In [0]:
#Import our model, different layers and activation function 
from tensorflow.keras.layers import Dense, LSTM, Reshape, BatchNormalization, Input, Conv2D, MaxPool2D, Lambda, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.activations import relu, sigmoid, softmax, selu, tanh
from keras.layers.advanced_activations import LeakyReLU
import tensorflow.keras.backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [0]:
#Model CRNN and BLSTM
from tensorflow.keras.layers import Layer, Conv2D, Multiply, Activation
BATCH_SIZE = 32


"""
Tensorflow Keras layer implementation of the gated convolution.
    Args:
        kwargs: Conv2D keyword arguments.
    Reference:
        T. Bluche, R. Messina,
        Gated convolutional recurrent neural networks for multilingual handwriting recognition.
        14th IAPR International Conference on Document Analysis andRecognition (ICDAR),
        p. 646–651, 11 2017.
"""


class GatedConv2D(Conv2D):
    """Gated Convolutional Class"""

    def __init__(self, **kwargs):
        super(GatedConv2D, self).__init__(**kwargs)

    def call(self, inputs):
        """Apply gated convolution"""

        output = super(GatedConv2D, self).call(inputs)
        linear = Activation("linear")(inputs)
        sigmoid = Activation("sigmoid")(output)

        return Multiply()([linear, sigmoid])

    def get_config(self):
        """Return the config of the layer"""

        config = super(GatedConv2D, self).get_config()
        return config


def bluche(input_size, d_model, learning_rate):
    """
    Gated Convolucional Recurrent Neural Network by Bluche et al.

    Reference:
        Bluche, T., Messina, R.:
        Gated convolutional recurrent neural networks for multilingual handwriting recognition.
        In: Document Analysis and Recognition (ICDAR), 2017
        14th IAPR International Conference on, vol. 1, pp. 646–651, 2017.
        URL: https://ieeexplore.ieee.org/document/8270042
    """

    input_data = Input(name="input", shape=input_size)
    cnn = Reshape((input_size[0] // 2, input_size[1] // 2, input_size[2] * 4))(input_data)

    cnn = Conv2D(filters=8, kernel_size=(2,1), strides=(2,1), padding="same", activation="relu")(cnn)
  
    cnn = Conv2D(filters=16, kernel_size=(2,1), strides=(1,1), padding="same", activation="relu")(cnn)
    cnn = GatedConv2D(filters=16, kernel_size=(3,1), strides=(1,1), padding="same")(cnn)
    batch = BatchNormalization()(cnn)

    cnn = Conv2D(filters=32, kernel_size=(3,1), strides=(1,1), padding="same", activation='relu')(batch)
    cnn = GatedConv2D(filters=32, kernel_size=(3,3), strides=(1,1), padding="same")(cnn)
    batch = BatchNormalization()(cnn)

    cnn = Conv2D(filters=64, kernel_size=(2,1), strides=(2,2), padding="same", activation="relu")(batch)
    cnn = GatedConv2D(filters=64, kernel_size=(2,2), strides=(1,1), padding="same")(cnn)
    batch = BatchNormalization()(cnn)

    cnn = Conv2D(filters=128, kernel_size=(3,1), strides=(2,2), padding="same", activation="relu")(batch)
    cnn = GatedConv2D(filters=128, kernel_size=(3,3), strides=(1,1), padding="same")(cnn)
    batch = BatchNormalization()(cnn)

    cnn = Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding="valid", activation="relu")(batch)
    cnn = MaxPool2D(pool_size=(2,1), padding='same')(cnn)
    cnn = MaxPool2D(pool_size=(1, 14))(cnn)
    batch = BatchNormalization()(cnn)
   
    
    cnn = MaxPool2D(pool_size=(63,1), padding='valid')(batch)
    squeezed = Lambda(lambda x: K.squeeze(x, 1))(cnn)



    shape = cnn.get_shape()
    blstm = Reshape((shape[1], shape[2] * shape[3]))(batch)

    blstm = Bidirectional(LSTM(units=256, return_sequences=True, dropout = 0.5))(blstm)

    blstm = Bidirectional(LSTM(units=256, return_sequences=True, dropout = 0.5))(blstm)
    output_data = Dense(units=d_model, activation="softmax")(blstm)

    if learning_rate is None:
        learning_rate = 1e-3

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    return (input_data, output_data, optimizer)

def ctc_loss_lambda_func(y_true, y_pred):
  """Function for computing the CTC loss"""

  if len(y_true.shape) > 1:
      y_true = tf.squeeze(y_true)

  # y_pred.shape = (batch_size, string_length, alphabet_size_1_hot_encoded)
  # output of every model is softmax
  # so sum across alphabet_size_1_hot_encoded give 1
  #               string_length give string length
  input_length = tf.math.reduce_sum(y_pred, axis=-1, keepdims=False)
  input_length = tf.math.reduce_sum(input_length, axis=-1, keepdims=True)

  # y_true strings are padded with 0
  # so sum of non-zero gives number of characters in this string
  label_length = tf.math.count_nonzero(y_true, axis=-1, keepdims=True, dtype="int64")

  loss = K.ctc_batch_cost(y_true, y_pred, input_length, label_length)

  # average loss across all entries in the batch
  loss = tf.reduce_mean(loss)

  return loss

In [0]:
#Loss function and call back
from tensorflow.keras.callbacks import CSVLogger, TensorBoard, ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
def get_callbacks(checkpoint, monitor="val_loss", verbose=0):
        """Setup the list of callbacks for the model"""

        callbacks = [
            ModelCheckpoint(
                filepath=checkpoint,
                monitor=monitor,
                save_best_only=True,
                save_weights_only=True,
                verbose=verbose),
            EarlyStopping(
                monitor=monitor,
                min_delta=1e-8,
                patience=10,
                restore_best_weights=True,
                verbose=verbose),
            ReduceLROnPlateau(
                monitor=monitor,
                min_delta=1e-8,
                factor=0.2,
                patience=5,
                verbose=verbose)
        ]

        return callbacks

In [0]:
#Summary model
input_size = (2048, 128, 1)
max_text_length = 140

outs = bluche(input_size, 140, 1e-3)

inputs, outputs, optimizer = outs

model = Model(inputs=inputs, outputs=outputs)
model.compile(optimizer=optimizer, loss=ctc_loss_lambda_func)
model.summary()

Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 2048, 128, 1)]    0         
_________________________________________________________________
reshape_24 (Reshape)         (None, 1024, 64, 4)       0         
_________________________________________________________________
conv2d_84 (Conv2D)           (None, 512, 64, 8)        72        
_________________________________________________________________
conv2d_85 (Conv2D)           (None, 512, 64, 16)       272       
_________________________________________________________________
gated_conv2d_56 (GatedConv2D (None, 512, 64, 16)       784       
_________________________________________________________________
batch_normalization_67 (Batc (None, 512, 64, 16)       64        
_________________________________________________________________
conv2d_86 (Conv2D)           (None, 512, 64, 32)       156

In [0]:
#Start train model
checkpoint = '/content/drive/My Drive/best.h5'
num_steps_train = tf.math.ceil(float(TRAIN_SAMPLES)/BATCH_SIZE)   
num_steps_test = tf.math.ceil(float(TEST_SAMPLES)/BATCH_SIZE)                        
epochs = 30
history = model.fit(train_ds,
                    steps_per_epoch = num_steps_train,
                    epochs=epochs,
                    validation_data=test_ds,
                    validation_steps=num_steps_test,
                    callbacks=get_callbacks(checkpoint=checkpoint))